In [1]:
%load_ext watermark

In [2]:
%watermark --gpu

GPU Info: 
  GPU 0: NVIDIA H100 80GB HBM3
  GPU 1: NVIDIA H100 80GB HBM3
  GPU 2: NVIDIA H100 80GB HBM3
  GPU 3: NVIDIA H100 80GB HBM3
  GPU 4: NVIDIA H100 80GB HBM3
  GPU 5: NVIDIA H100 80GB HBM3
  GPU 6: NVIDIA H100 80GB HBM3
  GPU 7: NVIDIA H100 80GB HBM3



In [3]:
%watermark

Last updated: 2023-05-29T16:44:04.329743-07:00

Python implementation: CPython
Python version       : 3.10.6
IPython version      : 8.13.2

Compiler    : GCC 11.3.0
OS          : Linux
Release     : 5.15.0-1017-nvidia
Machine     : x86_64
Processor   : x86_64
CPU cores   : 224
Architecture: 64bit



In [3]:
import numpy as np
import pandas as pd
import gc
import time

from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import KFold, StratifiedKFold
from tqdm import tqdm

import warnings
from time import time
import xgboost as xgb
xgb.__version__

'1.7.5'

In [4]:
%watermark --iversions

xgboost: 1.7.5
numpy  : 1.23.5
pandas : 1.5.3



In [5]:
ID = 'id'
TARGET = 'loss'
SEED = 0
DATA_DIR = "../input"

TRAIN_FILE = f"{DATA_DIR}/train.csv.zip"
TEST_FILE = f"{DATA_DIR}/test.csv.zip"
SUBMISSION_FILE = f"{DATA_DIR}/sample_submission.csv.zip"

In [6]:
shift = 200

train = pd.read_csv(TRAIN_FILE)
test = pd.read_csv(TEST_FILE)

y_train = np.log(train[TARGET].ravel()+shift)

train.drop([ID, TARGET], axis=1, inplace=True)
test.drop([ID], axis=1, inplace=True)

print(f"{train.shape},{test.shape}")
      
ntrain = train.shape[0]

(188318, 130),(125546, 130)


In [7]:
ntrain

188318

In [8]:
train_test = pd.concat((train, test)).reset_index(drop=True)

features = train.columns

cats = [feat for feat in features if 'cat' in feat]
for feat in cats:
    train_test[feat] = pd.factorize(train_test[feat], sort=True)[0]

print(train_test.head())

x_train = train_test.iloc[:ntrain,:]
x_test = train_test.iloc[ntrain:,:]

print(f"{train.shape},{test.shape}")

   cat1  cat2  cat3  cat4  cat5  cat6  cat7  cat8  cat9  cat10  ...     cont5  \
0     0     1     0     1     0     0     0     0     1      0  ...  0.310061   
1     0     1     0     0     0     0     0     0     1      1  ...  0.885834   
2     0     1     0     0     1     0     0     0     1      1  ...  0.397069   
3     1     1     0     1     0     0     0     0     1      0  ...  0.422268   
4     0     1     0     1     0     0     0     0     1      1  ...  0.704268   

      cont6     cont7    cont8    cont9   cont10    cont11    cont12  \
0  0.718367  0.335060  0.30260  0.67135  0.83510  0.569745  0.594646   
1  0.438917  0.436585  0.60087  0.35127  0.43919  0.338312  0.366307   
2  0.289648  0.315545  0.27320  0.26076  0.32446  0.381398  0.373424   
3  0.440945  0.391128  0.31796  0.32128  0.44467  0.327915  0.321570   
4  0.178193  0.247408  0.24564  0.22089  0.21230  0.204687  0.202213   

     cont13    cont14  
0  0.822493  0.714843  
1  0.611431  0.304496  
2  0.195

In [9]:
x_train.to_csv('../input/x_train_l.csv.zip', index=False, compression='zip')
x_test.to_csv('./inputx_test_l.csv.zip', index=False, compression='zip')

In [10]:
x_train.columns

Index(['cat1', 'cat2', 'cat3', 'cat4', 'cat5', 'cat6', 'cat7', 'cat8', 'cat9',
       'cat10',
       ...
       'cont5', 'cont6', 'cont7', 'cont8', 'cont9', 'cont10', 'cont11',
       'cont12', 'cont13', 'cont14'],
      dtype='object', length=130)

In [11]:
%%time
kf = KFold(5, shuffle=True, random_state=137)

for i, (train_index, test_index) in enumerate(kf.split(x_train,y_train)):

        print(i)
        xgtrain, xgval = x_train.loc[train_index], x_train.loc[test_index]
        y_tr, y_val = y_train[train_index], y_train[test_index]

        y_train_5 = []
        for ii in range(5):
            y_train_5.append(y_tr)

        trains = []
        for ii in range(5):
            trains.append(xgtrain)

        y_train_5 = np.hstack(y_train_5)

        X_num_train_5 = pd.concat(trains, ignore_index=True, axis=0)

        columns = x_train.columns
        
        X_num_train_5['target'] = y_train_5
        X_num_train_5 = X_num_train_5.sample(frac=1).reset_index(drop=True)
    
        xgval['target'] = y_val
        
        X_num_train_5.to_csv(f'../input2/xgtrain_fold_{i}_l.csv.gz', index=False, compression='gzip')
        xgval.to_csv(f'../input2/xgval_fold_{i}_l.csv.gz', index=False, compression='gzip')

0


<timed exec>:23: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
<timed exec>:26: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`


1


<timed exec>:23: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
<timed exec>:26: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`


2


...

3


<timed exec>:23: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
<timed exec>:26: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`


4


<timed exec>:23: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
<timed exec>:26: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`


CPU times: user 4min 41s, sys: 3.52 s, total: 4min 44s
Wall time: 4min 44s


In [12]:
X_num_train_5.shape

(753275, 131)

In [13]:
xgtrain.shape

(150655, 130)